In [3]:
class base:
    def __init__(self):
        print("base init")

class derived(base):
    pass

derived()

base init


In [ ]:
import numpy as numpy

class np_op:
    def __init__(self, *args, **kwargs):
        out = kwargs.pop("out")
        def func():
            self.op(*args, **kwargs, out=out.mem)
        def func_():
            out.mem = self.op(*args, **kwargs)
            self.func = func
        self.func = func_

    def __call__(self):
        self.func()

class mem:
    def __init__(self):
        self.mem = None

class np:
    class multiply(np_op):
        op = numpy.multiply



In [ ]:
from main import ModelSpecLarge, ModelSpecSmall, init_training_data
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle

class Test:
    def __init__(self,val):
        self.val = val


def func(self):
    print(self.val)

Test.func = func

(Test(1).func(), Test(2).func()

1
2


(None, None)

In [12]:
np.eye(3,3)[1,1]

np.float64(1.0)

## Testing

In [ ]:
network = ModelSpecSmall()
network.load_params("../params/small_50epochs_93percent")

In [ ]:
def file_it(path, skiplines=0):
    with open(path) as f:
        for _ in range(skiplines):
            f.readline()
        yield from f

it = map(
    lambda line: np.fromstring(line, sep=",", dtype=np.uint8),
    file_it("../data/test.csv", skiplines=1),
)

np.set_printoptions(formatter={"float": "{:.2f}".format})

In [ ]:
data = next(it)
n, pvec = network.feedforward(data/255)
print(n)
print(pvec)
fig,ax=plt.subplots()
ax.imshow(data.reshape((28,28)), cmap="gray", vmin = 0, vmax = 255)
ax.tick_params(axis='both', which='major', labelsize=14, length=10, width=2)
plt.show()

## Training

In [ ]:
network = ModelSpecSmall()
batchsize = 20
N_iterations, batch_it, valset = init_training_data("../data/train.csv", batchsize=batchsize, valsetsize=100, epochs=40)
network.init_training(batchsize=batchsize, validation_set=valset)

In [ ]:
for batch in batch_it:
    (training_loss, training_accuracy, validation_loss, validation_accuracy) = network.train_minibatch(batch)
    print(f"Loss: {validation_loss:10.4f}, Accuracy: {validation_accuracy:4.2f} (validation set)\r", end="")